In [20]:
%autoreload 2

In [30]:
from collections import defaultdict
from glob import glob
import pathlib
import random

from IPython.core.display import display, HTML, Markdown
import pandas as pd
from tqdm.notebook import tnrange, tqdm

from mutation_experiments import *
import ast_printer

In [22]:
args = parser.parse_args([])

grammar = open(args.grammar_file).read()
grammar_parser = tatsu.compile(grammar) 

asts = load_asts(args, grammar_parser)

In [46]:
NUM_GAMES = 100

regrwoth_mutated_asts = []

for seed in tnrange(NUM_GAMES):
    validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
    sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)
    random.seed(seed)
    try:
        mutated_ast = sampler.sample_regrowth()
    except SamplingFailedException:
        continue
    regrwoth_mutated_asts.append((mutated_ast, seed))
        


In [47]:
NUM_GAMES = 100
START = 100
single_step_mutated_asts = []

for seed in tnrange(NUM_GAMES):
    validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
    sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)
    random.seed(START + seed)
    try:
        mutated_ast = sampler.sample_single_step()
    except:
        continue
    single_step_mutated_asts.append((mutated_ast, START + seed))
        


In [115]:
DATA_COLUMNS = ('date', 'setup', 'gameplay', 'scoring', 'difficulty', 'first_score', 'strategy', 'technical_difficulties', 'other_comments')
PARTICIPANT_DATA_DFS = dict()
for p in glob('./data/*'):
    path = pathlib.Path(p)
    df = pd.read_csv(path)
    df.columns = DATA_COLUMNS
    PARTICIPANT_DATA_DFS[path.name.replace(path.suffix, '').replace('_', '-')] = df


def display_description(game_or_name):
    if isinstance(game_or_name, tuple):
        name = game_or_name[1].game_name

    else:
        name = game_or_name

    last_dash_idx = name.rfind('-')
    name, game_id = name[:last_dash_idx], int(name[last_dash_idx + 1:])
    original_data = PARTICIPANT_DATA_DFS[name].iloc[game_id - 2,1:4]  # -2 because my count starts at 2 and the first row is 0

    display(Markdown('### Original Description:'))
    for header, data in original_data.items():
        display(Markdown(f'**{header.capitalize()}:** {data}'))

    display(Markdown('----------'))
    display(Markdown('### Game:'))


In [116]:
def ast_to_buffer(ast, html=True):
    ast_printer.reset_buffers(True)
    context = dict()
    if html:
        context['html'] = True
    ast_printer.pretty_print_ast(ast, context=context)


def new_ast_to_buffer(new_asts, index, html=True):
    mutated_ast, _ = new_asts[index]
    ast_to_buffer(ast, html=html)


def visualize_ast(ast, html=True, original_description=True):
    ast_to_buffer(ast, html=html)
    out_str = '\r\n'.join(ast_printer.BUFFER)

    if original_description:
        display_description(ast)

    if html:
        display(HTML(out_str))
    else:
        print(out_str)


def visualize(new_asts, index, html=True, original_description=True):
    mutated_ast, seed = new_asts[index]
    print(f'With random seed {seed}, mutated:\r\n')
    visualize_ast(mutated_ast, html=html, original_description=original_description)
    


# Games generated with full regrwoth

I'll pring out some interesting games, and try to explain the game and the change that was made. 

The color-coding on these is perhaps not the most informative, but:
* <span style="color: #AA00FF;">Purple:</span> the original node substituted in
* <span style="color: #FF6A00;">Orange:</span> any nodes sampled under the original node from another game
* <span style="color: #00FF80;">Green:</span> nodes from the substituted subtree (the purple one) where we replaced at least one value under the node, but not the entire node.

**TL;DR: if it's colored, it was't in the original game**


In the game below, it changed the quantification and type from `?g - game_object` to `?g - dodgeball`.

Originally, that clause was marking keeping the beachball (`?b`) in the air for 20 counts -- by providing it's not or touching any other object.

Now it stipulates the beachball can't touch or be in the dodgeball, which is rather more trivial


In [117]:
visualize(regrwoth_mutated_asts, 0)

With random seed 0, mutated:



### Original Description:

**Setup:** The two large triangular ramps are placed next to each other in the middle of the room.

**Gameplay:** The two large triangular ramps are placed next to each other in the middle of the room. The player must slide through them, sit in one of the chairs, spin 4 times, throw all the 3 dodgeballs into the hexagonal bin and then keep the beachball in the air for 20 seconds. The circuit will be timed by the cellphone on the desk. 

**Scoring:** Circuit done under 40 seconds: 20 points 
Circuit done in 50 seconds:18 points
Circuit done in 60 seconds: 15 points
Circuit done over 60 seconds:13 points

----------

### Game:

This is another similar change. Originally, this quantified `?b - bed` like in the previous game, to mark that the player should make the throw from the bed. 

With this change, it marks that the player should make the throw while standing on a block, which is cute.

In [118]:
visualize(regrwoth_mutated_asts, 2)

With random seed 2, mutated:



### Original Description:

**Setup:** 2 chairs,pillow,doggie bed, bed

**Gameplay:**  you have to hit the 2 chairs with the doggie bed and the pillow while sitting on the bed

**Scoring:** 20 score each pillow or doggie bed hit the 2chairs 10 times in a row
 

----------

### Game:

Funny failure mode -- regardless of what this attempted to convey originally, this can never occur, obviously.

In [119]:
visualize(regrwoth_mutated_asts, 4)

With random seed 4, mutated:



### Original Description:

**Setup:** 2 tall cylindrical blocks, basketball, teddy bear

**Gameplay:** place 2 tall cylindrical blocks as a target to throw the basketball in between them and after passing between them there will be the teddy bear.

**Scoring:** 5 rounds - 1 basketball roll each round. 15 points if you roll the ball between the 2 tall cylindrical blocks without hitting them, and then hitting the teddy bear. You lose 5 points for each tall cylindrical block you hit.

----------

### Game:

Another amusing failure mode -- ths mutation performed substituted in the predicate `(on ?1 ?2)`, and renaming variables to match the local context created an identical predicate to the one that's already there. 

In [120]:
visualize(regrwoth_mutated_asts, 9)

With random seed 9, mutated:



### Original Description:

**Setup:** In this game all the objects start at the same point design in the picture. All you need is the 3 golfballs, 3 tall cylindrical blocks, 3 cube blocks, 3 short cylindrical blocks, 3 pyramidal blocks, 3 triangular blocks, 3 tall cylindrical blocks, 3 flat rectangular blocks, 3 bridge blocks,  2 large triangular ramps, curved wooden ramp,  3 dodgeballs, beachball.

**Gameplay:** In this game you will have to make an bowling alley with the object provided. Then you use the balls to play in your bowling alley. This give infinite solutions to the game.

**Scoring:** If you use the 3 dodgeballs to strike the blocks you will get lesser points than when you use the 3 golfballs and more than when you use the beachball. Striking the bigger objects give you lesser points than if you strike bigger objects. Striking with the golfballs will multiply by 3 the score you got by striking the objects. If you strike using the dodgeballs it will multiply by 2 and the beachball by 1. Striking the short cylindrical blocks will give 5 points each. Striking the pyramid and bigger cylindrical block will give you 3 points. Striking 3 flat rectangular blocks and using the 3 bridge blocks will give you 1 points each. The final scores is calculated by the strike points and the multiplying that to the score of each ball.

----------

### Game:

Changes the scoring from awarding five points for each succesful throw (`throwLandsInTarget`) to five points for each throw. Makes the game easier?

In [121]:
visualize(regrwoth_mutated_asts, 11)

With random seed 11, mutated:



### Original Description:

**Setup:** 3 golfballs, a large triangular ramp, the 3 triangular blocks and the 3 bridge block

**Gameplay:** First, it'd be ideal to clear out the rug. Second, you set up the 3 triangular blocks so that they make a triangular shape from above (as big as you want it to be). That will be your main target. Then, you put the bridge blocks around the shape, a fair distance from it, creating sort of a perimeter; that's another target. Now that you've got your targets, you imagine where you want to throw the golf balls from, and put the large triangular ramp between you and the targets. Now the only thing left to do is to throw them as if they were bowling balls, and using the ramp, make it land in one of the goals.

**Scoring:** You only get 3 throws, and a throw that lands in the main target gives you 5 points. A throw that misses the target, but gets caught by the perimeter will give you 2 points. Otherwise, if the ball leaves the perimeter, you'll get no points.

----------

### Game:

Another amusing failure mode that occurs a bunch -- a predicate being called with the wrong number of arguments as result of a mutation.

Semantically, I considered `(agent_holds ?1)` to accept one argument, and `(on ?1 ?2)` to accept two, and here they ended up flipped.

In [122]:
visualize(regrwoth_mutated_asts, 15)

With random seed 15, mutated:



### Original Description:

**Setup:** I would perch the hexagonal bin on top of the shelve furthest from the desk and sit on one of the chairs, looking at the desk

**Gameplay:** The game would consist on trying to get the basketball into the hexagonal bin by throwing it over your head and looking upside down, with your neck hanging from the back of the chair

**Scoring:** You would get a point for every time the ball got into the bin

----------

### Game:

This game originally was something like "consturct a a tunnel from the flat and bridge blocks, then throw the ball under the bridge.

The change essentially makes it such that you have to construct a tunnel from the ramp and bridge blocks, but it doesn't enforce the first setup term, which was designed to make sure this bridge is constructed on the floor.

In [123]:
visualize(regrwoth_mutated_asts, 25)

With random seed 26, mutated:



### Original Description:

**Setup:** 2 bridge blocks, 2 flat rectangular blocks, basketball

**Gameplay:** create tunnels with the brigde blocks and throw the balls

**Scoring:** 1 point for teach

----------

### Game:

Changes the terminal conditions, which were originally `chairStoppedRotating` happening once (that is, the chair stops rotating), to `objectLandsOnRotatingChair` happening once, that is, ending at the first succesful shot. 

Not nonsense, but doesn't make too much sense either.

In [124]:
visualize(regrwoth_mutated_asts, 27)

With random seed 28, mutated:



### Original Description:

**Setup:** One of the chairs has to be still in the same spot rotating.

**Gameplay:** The game uses the chair rotating as it was prepared you will have to throw items into the rotating in the chair forming a stack of items. it ends when an item falls from the chair.

**Scoring:** 10 points per item and 10 points per 5 centimeters.

----------

### Game:

Changes this game from balancing on the large triangular ramp (what it was before) to balancing on a cube block, which sounds like it would be rather harder?

In [125]:
visualize(regrwoth_mutated_asts, 40)

With random seed 41, mutated:



### Original Description:

**Setup:** Prepare one of the large triangular ramps and place it in the doggie bed.

**Gameplay:** Try to stand on the hypotenuse side of the large triangular ramp while balancing it on its right-angled edge.

**Scoring:** Have a stopwatch in one hand while balancing and time it. Seconds balanced = score.

----------

### Game:

Game changed from bowling a dodgeball to bowling a cube block

In [126]:
visualize(regrwoth_mutated_asts, 42)

With random seed 43, mutated:



### Original Description:

**Setup:** The curved wooden ramp is placed in the center of the room and so is the hexagonal bin, next to it.

**Gameplay:** The goal in this game is to get the dodgeballs to go through the curved wooden ramp into the hexagonal bin, like bowling with a twist.

**Scoring:** Everytime the player scores the dodgeball into the hexagonal bin, wins 5 points.

----------

### Game:

Another minor semantic change, from bouncing a ball off the wall and catching it, to bouncing a ball off the curved wooden ramp and catching it.

In [127]:
visualize(regrwoth_mutated_asts, 53)

With random seed 55, mutated:



### Original Description:

**Setup:** All that is need for this game is one of the dodgeballs and the wall.

**Gameplay:** The player can bounce the dodge ball off of the wall and catch it. 

**Scoring:** Each time the player catches the ball, it's one point. Each time they step back further and the points that can be gained increase by one. 

----------

### Game:

# Games generated by making a single step

In these the color scheme is simpler -- the mutation is in blue (first color in a equential colormap, assuming I might want to eventually mark each mutation, or something)

Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

In [128]:
visualize(single_step_mutated_asts, 3)

With random seed 103, mutated:



### Original Description:

**Setup:** bed
hexagonal bin on the bed
curved wooden ramp about 2m to the bed
3 dodgballs

**Gameplay:** On the bed i'd place the hexagonal bin. About 2m of the bed i'd place the curved wooden ramp. Then, close to the wall of the opposite side of the bed i'd throw 1 dodge ball at a time through the curved wooden ramp trying to place the dodgball in the hexagonal bin

**Scoring:** for each dodgeball place inside the bin i'd get 1 point

----------

### Game:

Another minor semantic difference -- from ending the game after three roll attempts, to ending it after three succesful rolls.

In [129]:
visualize(single_step_mutated_asts, 6)

With random seed 106, mutated:



### Original Description:

**Setup:** nan

**Gameplay:** The player will use all of the different blocks available to create a course for the golf balls that starts from the bottom self that is located in front of the bed and leads to the hexagonal bin (the inside of which, is the finish-line). The player has 3 attempts (using the 3 gold balls) to perfect his course. For each attempt the player will also be using the cellphone to time how long it takes for the golf ball to make it inside the hexagonal bin or untill the golf ball falls down to the floor.

**Scoring:** The score is calculated by adding the seconds timed in all 3 attempts and multiplying the total number of seconds by 10.

----------

### Game:

A small change to the scoring -- if you manage to knock off the desk lamp at least once, you get an extra point for each time you manage to knock off the desktop.

In [130]:
visualize(single_step_mutated_asts, 7)

With random seed 107, mutated:



### Original Description:

**Setup:** Desktop computer, desk lamp, CD, pillow, beachball and dodgeball.

**Gameplay:** In this game you need to knock down the objects that are on the desk with three projectile objects at your disposal; a pillow, a beachball and a dodgeball. You have to knock down a desktop computer, a desk lamp, and a CD. 

**Scoring:** Knocking down the desktop computer gives you 5 points, the desk lamp 10 points and the CD 15 points.

----------

### Game:

I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

In [131]:
visualize(single_step_mutated_asts, 19)

With random seed 121, mutated:



### Original Description:

**Setup:** All that is need for this game is one of the dodgeballs and the wall.

**Gameplay:** The player can bounce the dodge ball off of the wall and catch it. 

**Scoring:** Each time the player catches the ball, it's one point. Each time they step back further and the points that can be gained increase by one. 

----------

### Game:

Throw the cellphone such that it is adjacent to the doggie bed, rather than such that it lands on it. Cute and valid.

In [132]:
visualize(single_step_mutated_asts, 21)

With random seed 123, mutated:



### Original Description:

**Setup:** Textbook, laptop, cellphone and doggie bed. 

**Gameplay:** The game consists of throwing objects on the doggie bed. You have three objects to throw a textbook, laptop and a cellphone. Landing any of the aforementioned objects on the bed gives you points; 15 points the cellphone, 10 the textbook and 5 the laptop.

**Scoring:** Landing the cellphone on the bed gives you 15 points, the textbook 10 points and the laptop 5 points.

----------

### Game:

Originally, that preference asked to to the chair with the pillow. Now it asks that after throwing the pillow, and before the pillow stops moving (or the agent picks it up again), the chair will move. Presumably this could happen because the agent moves it, or the pillow hits it. Totally valid.

In [133]:
visualize(single_step_mutated_asts, 24)

With random seed 126, mutated:



### Original Description:

**Setup:** 2 chairs,pillow,doggie bed, bed

**Gameplay:**  you have to hit the 2 chairs with the doggie bed and the pillow while sitting on the bed

**Scoring:** 20 score each pillow or doggie bed hit the 2chairs 10 times in a row
 

----------

### Game:

I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`


Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

In [134]:
visualize(single_step_mutated_asts, 38)

With random seed 145, mutated:



### Original Description:

**Setup:** The 3 cube blocks, 3 tall cylindrical blocks and the 3 short cylindrical blocks should be moved to the center of the room in an area that the player can see the clock. 

**Gameplay:** In this game, the player can use the clock to keep track of time. Each round, the player has one minute to build a tower using the blocks as tall as they can without it falling over.

**Scoring:** Each block that is added to the tower before the time runs out and it falls over is one point gained by the player. 

----------

### Game:

This change to the setup is mostly coherent (even if one of the terms is repeated, although with the arguments in the other order), but I don't know how much semantic sense it makes. If I were to try to describe it in natural language:

"Place two blocks next to the chair and to each other. One of those blocks should be on another block. Place yet another block next to the block you put on top, and place another block next to the block you just placed. Finally, place the last block on the floor, next to the block you placed other blocks on."

In [135]:
visualize(single_step_mutated_asts, 39)

With random seed 146, mutated:



### Original Description:

**Setup:** 6 cube blocks, in the middle of the room
1 dodgball
1 chair

**Gameplay:** i'd place the 6 cube blocks in the middle of the room, 1 on top of other, in pairs, making 3 collumns, 1 up front and 2 behind it. Then i'd be sitted in 1 chair and would try to strike the 6 cube blocks with 1 dodgeball to make them all be at the same level (all 6 cube blocks on the floor)

**Scoring:** Being 3 collumns, there would be 3 top cubes that could be dethroned to the floor; each cube that fell from the top of the collumn would make 1 point

----------

### Game:

Slightly changed how the preference is specified -- previously it was `(hold (and (not (agent_holds ?d)) (in_motion ?d))) `, that is, the agent doesn't touch the ball while it's in motion, and now it specifies that the block can't move while the ball is in motion, until they touch. 

In other words, previously you could move the block to make the moving ball touch it, and now you can make contact with the ball (while not stopping its motion) but not the block.

In [136]:
visualize(single_step_mutated_asts, 40)

With random seed 147, mutated:



### Original Description:

**Setup:** The 6 cube blocks will have to be placed in different places all around the room(ex. inside hexagonal bin, in front of curved wooden ramp, one on top and one on bottom selves that are located above the hexagonal bin, to the corner of the room at the right side of the desk and on top on the chair that is located at the left of the desk) Finally, the player will use the 2 dodgeballs to play the game so he should retrieve them from the self.

**Gameplay:** The player will sit on the bed with the 2 dodgeballs and his cellphone. He will then set a 60 second countdown on his cellphone. Once the countdown begins, the player will attempt to hit the 6 cube blocks by throwing the 2 dodge balls. If the dodge ball makes contact with the cube, then the player can get up from the bed to retrieve the dodgeball. The player can only throw the dodgeball if he is sitting on the bed. The player is also not allowed to move the cube blocks by hand (if a cube block falls from the self for example, it shall stay there for the rest of the game). The games ends once the countdown is finished. The player earns score points by hitting the cube blocks with the 2 dodgeballs.

**Scoring:** If the thrown dodgeball simply touches the cube block, then the player earns 5 points.
If the cube block flips or move a long distance when hit by the dodge ball, then the player earns 10 points.
Only the first time that a cube block gets hit counts towards the score.

----------

### Game:

Changed from a "throw after dribbling game" to simpler "throw in the bin" game.

In [137]:
visualize(single_step_mutated_asts, 42)

With random seed 150, mutated:



### Original Description:

**Setup:** alarm clock, basketball, hexagonal bin
2 cube blocks, 2 short cylindrical blocks, placed in a square, about 2m in distance between every one of this 4 blocks
basketball to drible

**Gameplay:** I'd place 2 cube blocks and 2 short cylindrical blocks in a square, about 2m in distance between every one of this 4 blocks, in the middle of the room. I'd put the hexagonal bin slightly further from the wall that has the empty shelves. Then, i'd use the clock to time 2 minutes that the game would last. I'd drible with the basketball around each of the 4 blocks used and shoot the basketball to the hexagonal bin as many times I could during the 2 minutes.

**Scoring:** Each shot getting in the hexagonal bin would count as 1 point

----------

### Game:

Originally that predicate was a much more complicated one to count an object landing on the chair, or on another object on the chair: 
```
(or 
    (on ?c ?o)
    (exists (?o2 - game_object) (and 
        (not (= ?o ?o2))
        (in_building ?o2)
        (on ?o2 ?o)
    ))
)
```
Now, it counts simpler occurences -- either the chair ending up on object, which is a little nonsensical, or an object touching the chair, which makes more sense.

In [138]:
visualize(single_step_mutated_asts, 56)

With random seed 168, mutated:



### Original Description:

**Setup:** One of the chairs has to be still in the same spot rotating.

**Gameplay:** The game uses the chair rotating as it was prepared you will have to throw items into the rotating in the chair forming a stack of items. it ends when an item falls from the chair.

**Scoring:** 10 points per item and 10 points per 5 centimeters.

----------

### Game:

Changed from the agent dribbling around a cube block, to the agent dribbling around those items quantified.

In [139]:
visualize(single_step_mutated_asts, 62)

With random seed 174, mutated:



### Original Description:

**Setup:** 2 dodgeballs, hexagonal bin, 2 chairs 

**Gameplay:** 
insert the dodgeballs into the hexagonal bin, and then use the chair to move and make more dificult to insert the balls.

**Scoring:** each basket is worth 2 points

----------

### Game:

# Second mutation of interesting games

In [140]:
INDICES_TO_MUTATE = (3, 6, 7, 19, 21, 24, 38, 39, 40, 42, 56, 62,)
NUM_MUTATIONS_TO_GENERATE = 10
MAX_GENERATION_ATTEMPTS = 100

second_step_mutations_dict = defaultdict(list)

validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)

for idx in tqdm(INDICES_TO_MUTATE):
    src_ast = single_step_mutated_asts[idx][0]
    seed = -1
    while seed < MAX_GENERATION_ATTEMPTS and len(second_step_mutations_dict[idx]) < NUM_MUTATIONS_TO_GENERATE:
        seed += 1
        random.seed(seed)
        try:
            src_ast_copy = copy_ast(grammar_parser, src_ast)
            copy_mutation_tags(src_ast, src_ast_copy)
            mutated_ast = sampler.sample_single_step(src_ast_copy, n_mutations=1)
        except:
            continue
        second_step_mutations_dict[idx].append((mutated_ast, seed))
        




# Visualizing choice second mutations
The second mutation will be in orange

**Original note**: Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

**Second mutation**: changed from landing the ball in/on the hexagonal bin, to landing the dodgeball on the golfball, which is rather difficult (if not impossible), but still valid

In [141]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[0]], 4)

With random seed 4, mutated:



### Original Description:

**Setup:** bed
hexagonal bin on the bed
curved wooden ramp about 2m to the bed
3 dodgballs

**Gameplay:** On the bed i'd place the hexagonal bin. About 2m of the bed i'd place the curved wooden ramp. Then, close to the wall of the opposite side of the bed i'd throw 1 dodge ball at a time through the curved wooden ramp trying to place the dodgball in the hexagonal bin

**Scoring:** for each dodgeball place inside the bin i'd get 1 point

----------

### Game:

**Original note**: Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

**Second mutation**: changed from landing on the in/on the hexagonal bin, to catching it and holding it again. Totally valid.

In [142]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[0]], 6)

With random seed 6, mutated:



### Original Description:

**Setup:** bed
hexagonal bin on the bed
curved wooden ramp about 2m to the bed
3 dodgballs

**Gameplay:** On the bed i'd place the hexagonal bin. About 2m of the bed i'd place the curved wooden ramp. Then, close to the wall of the opposite side of the bed i'd throw 1 dodge ball at a time through the curved wooden ramp trying to place the dodgball in the hexagonal bin

**Scoring:** for each dodgeball place inside the bin i'd get 1 point

----------

### Game:

**Original note:** I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

**Second mutation:** this changes the objects the game is played with again, from the dodgeball and bin to the basketball and bin

In [143]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[3]], 1)

With random seed 1, mutated:



### Original Description:

**Setup:** All that is need for this game is one of the dodgeballs and the wall.

**Gameplay:** The player can bounce the dodge ball off of the wall and catch it. 

**Scoring:** Each time the player catches the ball, it's one point. Each time they step back further and the points that can be gained increase by one. 

----------

### Game:

**Original note:** I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

**Second mutation:** Another minor change that seems to work -- instead of measuring the distance to the agent when the ball is caught, measure the distance to the wall

In [144]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[3]], 6)

With random seed 6, mutated:



### Original Description:

**Setup:** All that is need for this game is one of the dodgeballs and the wall.

**Gameplay:** The player can bounce the dodge ball off of the wall and catch it. 

**Scoring:** Each time the player catches the ball, it's one point. Each time they step back further and the points that can be gained increase by one. 

----------

### Game:

**Original note:** I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

**Second mutation:** Changed from bouncing the ball off the wall to landing it on the bin.

In [145]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[3]], 7)

With random seed 8, mutated:



### Original Description:

**Setup:** All that is need for this game is one of the dodgeballs and the wall.

**Gameplay:** The player can bounce the dodge ball off of the wall and catch it. 

**Scoring:** Each time the player catches the ball, it's one point. Each time they step back further and the points that can be gained increase by one. 

----------

### Game:

**Original note:** Throw the cellphone such that it is adjacent to the doggie bed, rather than such that it lands on it. Cute and valid.

**Second mutation:** Added the constraint that the cellphone must be thrown with the agent's eyes closed.

In [146]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[4]], 1)

With random seed 1, mutated:



### Original Description:

**Setup:** Textbook, laptop, cellphone and doggie bed. 

**Gameplay:** The game consists of throwing objects on the doggie bed. You have three objects to throw a textbook, laptop and a cellphone. Landing any of the aforementioned objects on the bed gives you points; 15 points the cellphone, 10 the textbook and 5 the laptop.

**Scoring:** Landing the cellphone on the bed gives you 15 points, the textbook 10 points and the laptop 5 points.

----------

### Game:

**Original comment:** I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`. Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

**Second mutation:** This time it changes the rules of consturction again, starting from a block (and counting at most one block), but then scoring for every ball (dodgeball or beachball) in the structure.

In [147]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[6]], 0)

With random seed 0, mutated:



### Original Description:

**Setup:** The 3 cube blocks, 3 tall cylindrical blocks and the 3 short cylindrical blocks should be moved to the center of the room in an area that the player can see the clock. 

**Gameplay:** In this game, the player can use the clock to keep track of time. Each round, the player has one minute to build a tower using the blocks as tall as they can without it falling over.

**Scoring:** Each block that is added to the tower before the time runs out and it falls over is one point gained by the player. 

----------

### Game:

**Original comment:** I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`. Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

**Second mutation:** Another change to the construction rules, this time to start building from a ball touching the floor.

In [148]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[6]], 2)

With random seed 3, mutated:



### Original Description:

**Setup:** The 3 cube blocks, 3 tall cylindrical blocks and the 3 short cylindrical blocks should be moved to the center of the room in an area that the player can see the clock. 

**Gameplay:** In this game, the player can use the clock to keep track of time. Each round, the player has one minute to build a tower using the blocks as tall as they can without it falling over.

**Scoring:** Each block that is added to the tower before the time runs out and it falls over is one point gained by the player. 

----------

### Game:

**Original comment:** I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`. Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

**Second mutation:** This time the initial block (which is the base of the construction, theoretically) must be adjacent to the agent and touched by the agent when the game ends. 

In [149]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[6]], 4)

With random seed 5, mutated:



### Original Description:

**Setup:** The 3 cube blocks, 3 tall cylindrical blocks and the 3 short cylindrical blocks should be moved to the center of the room in an area that the player can see the clock. 

**Gameplay:** In this game, the player can use the clock to keep track of time. Each round, the player has one minute to build a tower using the blocks as tall as they can without it falling over.

**Scoring:** Each block that is added to the tower before the time runs out and it falls over is one point gained by the player. 

----------

### Game:

In [150]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[4]], 2)

With random seed 2, mutated:



### Original Description:

**Setup:** Textbook, laptop, cellphone and doggie bed. 

**Gameplay:** The game consists of throwing objects on the doggie bed. You have three objects to throw a textbook, laptop and a cellphone. Landing any of the aforementioned objects on the bed gives you points; 15 points the cellphone, 10 the textbook and 5 the laptop.

**Scoring:** Landing the cellphone on the bed gives you 15 points, the textbook 10 points and the laptop 5 points.

----------

### Game:

# Selected games with longer mutations

They tend to be less coherent, so I didn't prioritize them early, but here are a couple:

Here's one where an entire (then ...) block is replaced, but the main problem arises from the variable replacement. Any variable defined in the original block that doesn't exist in the current block (say, `?b`) gets randomly replaced with a variable that is valid in the current context (say, `?h`).

So here, we end up with a game that starts with the agent holding a dodgeball, and then continue to the agent not holding the dodgeball while the hexagonal bin is in motion, and ending with the wall not being in motion. That's not quite right. 

In [158]:
visualize(single_step_mutated_asts, 8)

With random seed 109, mutated:



### Original Description:

**Setup:** Place the hexagonal bin at one end of the room 

**Gameplay:** Use the dodgeball to aim into the hexagonal
Bin from the other side of the room. Like shooting hoops. 

**Scoring:** 1 point for every dodgeball in the hexagonal bin. 

----------

### Game:

Here's one where we replace effectively the entire setup, but the new setup makes no sense context in the of the gameplay constraints -- there's no mention of the large triangular ramps anywhere in gameplay.

In [174]:
visualize(single_step_mutated_asts, 64)

With random seed 176, mutated:



### Original Description:

**Setup:** Set up, in the room's open space, the curved wooden ramp to be directly facing the hexagonal bin. Place the dodgeball roughly 1 metre away from the base of the curved wooden ramp. Use the textbook to mark a distance for the dodgeball.

**Gameplay:** Attempt to kick the dodgeball into the hexagonal bin, using the ramp as a way to lift the ball. Do this 10 times.

**Scoring:** Count how many times, out of 10, you manage to get the dodgeball into the hexagonal bin.

----------

### Game:

Here's another example -- the preference `ballEndedonCD` ended up being repurposed for the agent throwing a beachball and knocking over a block (agent starts holding ball while block is on the floro, and then the block is touched by the ball and becomes in motion).

I guess that it works by itself, but doesn't necesarily make semantic sense within the context of the rest of the game? Definitely valid, though.

In [182]:
visualize(single_step_mutated_asts, 18)

With random seed 120, mutated:



### Original Description:

**Setup:** 3 golfballs, CD, poster, rug

**Gameplay:** This game is really simple, place your rug away from you, put your poster on it, and then put your CD in the middle of your poster. Your goal is to roll a ball into that cd. You have 3 balls, and 3 areas that give you points. You can roll 1 ball only once, however when your first ball is on the rug, you can move it by hitting it with the next ball, so it moves into the poster. The game ends when you already used your 3 balls.

**Scoring:** If the ball is on the tug, you gain 1 point, if the ball is on the poster, you gain 2 points, and if your ball is on CD you gain 3 points. You gain 0 points for the ball that is on the floor instead of this 3 areas.

----------

### Game:

Another example of replacing an entire preference (then ...) block going awry. It's broken from the first term, which claims the preference starts from a state where the agent holds the ramp and is standing on the ramp...

In [188]:
visualize(single_step_mutated_asts, 60)

With random seed 172, mutated:



### Original Description:

**Setup:** nan

**Gameplay:** Sort the geometrical shapes (pyramidal blocks, bridge blocks, golfballs, tall cylindrical blocks, triangle ramps, cube blocks, tall cylindrical blocks, short cylindrical blocks, flat rectangular blocks, dodgeballs) by shape and colour, from smallest to largest and by colour in a particular order decided beforehand; the order can differ from game to game, but it must be decided before starting the game.

**Scoring:** For each item correctly placed before the time runs out the player gets 1 point. The maximum allowed time is 2 minutes and the aim is to sort as many items as possible in that time and when the participant sorts all of the items in less than 2 minutes, he can try to beat his score by decreasing the allowed time.

----------

### Game:

# Debugging


In [171]:
import numpy as np
mutations = [find_mutations(ast, should_print=False)[0][0] for ast in single_step_mutated_asts]
np.argsort(mutations)

array([ 6, 50, 64, 63, 49, 84, 47, 33, 37, 85,  3, 54,  7, 18, 26, 23, 39,
       60, 32, 66, 42, 67, 68, 44,  8,  4, 81, 25, 36, 41, 83, 51, 45, 62,
       28, 78, 22, 19, 17, 13, 69, 31, 55, 79, 53, 72, 73, 74, 38, 76, 46,
       75, 12, 15, 24, 48, 20, 35,  9,  1,  2, 71, 77, 10, 11, 21, 27, 29,
       59, 57, 56, 34, 52, 80, 65, 43, 82, 87, 61, 86, 40, 16,  5, 70, 58,
       30, 14,  0])

7 {'variables': [{'var_names': ['?r'], 'var_type': 'large_triangular_ramp', 'parseinfo': [None, 'variable_type_def', 137, 163, 0, 0]}], 'parseinfo': [None, 'variable_list', 135, 164, 0, 0], 'mutation': 0}


[(7,
  {'variables': [{'var_names': ['?r'], 'var_type': 'large_triangular_ramp', 'parseinfo': [None, 'variable_type_def', 137, 163, 0, 0]}], 'parseinfo': [None, 'variable_list', 135, 164, 0, 0], 'mutation': 0})]

In [32]:
new_ast_to_buffer(single_step_mutated_asts, 44)
[line for line in ast_printer.BUFFER if 'color' in line]

['<div style="margin-left: 60px"><span style="">(exists</span> <span style="color: #1f77b4">(<span style="">?r - large_triangular_ramp</span>)</span> </div>']

In [33]:
for idx, ast in enumerate(single_step_mutated_asts):
    new_ast_to_buffer(single_step_mutated_asts, idx)
    if sum(['color' in line for line in ast_printer.BUFFER]) == 0:
        print(idx)

In [ ]:
random.seed(0)
src_ast = single_step_mutated_asts[INDICES_TO_MUTATE[0]][0]
src_ast_copy = copy_ast(grammar_parser, src_ast)
copy_mutation_tags(src_ast, src_ast_copy)